In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
import torch
from catalyst.utils import set_global_seed, prepare_cudnn
from catalyst.dl import Runner
from src.nn import BACTERIA, DiceLoss, get_model
prepare_cudnn(deterministic=True)
set_global_seed(0)

In [ ]:
criterion = DiceLoss()

class CustomRunner(Runner):

    def _handle_batch(self, batch):
        global criterion
        x, y, z = batch
        is_train = self.state.stage_name == 'train'
        with torch.set_grad_enabled(is_train):
            y_pred = self.model(x)
            loss = criterion(y_pred, y)
            self.state.batch_metrics = {
                'loss': loss,
            }
            if is_train:
                loss.backward()
                self.state.optimizer.step()
                self.state.optimizer.zero_grad()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
torch.cuda.empty_cache()
model = get_model()
model.train()
dataset = BACTERIA()
loaders = next(dataset.crossval(kfold=4, batch_size=9))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
runner = CustomRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    num_epochs=100,
    logdir='./logs/segmentation',
    verbose=True,
)

In [ ]:
! zip -r logs/segmentation.zip logs/segmentation
! git add src logs
! git commit -m 'Changed from Colab'
! git push -u origin master